In [1]:
from datasets import load_dataset

dataset = load_dataset("medical_questions_pairs")

Using custom data configuration default
Reusing dataset medical_questions_pairs (/home/parker/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae)


In [2]:
dataset = dataset["train"].train_test_split(test_size=0.2)
train = dataset["train"]
test = dataset["test"]

Loading cached split indices for dataset at /home/parker/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae/cache-ddbba2ce54e35142.arrow and /home/parker/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae/cache-4f60af92002c21dd.arrow


In [3]:
import pandas as pd

train = train.to_pandas()
test = test.to_pandas()

In [4]:
train.head()

,dr_id,label,question_1,question_2
0,3,1,Is it contraindicated to insert a urinary cath...,I am looking for some information- Is it safe ...
1,8,1,How do I tell the difference between an allerg...,What are the symptoms that differentiate betwe...
2,1,1,Can coarctation of the aorta cause poor growth...,"I am 4' 8"". My mom said that I have a birth de..."
3,9,0,I had a sinus infection 3 weeks ago and i was ...,My sinuses have been blocked due to allergies ...
4,9,0,Clicking sound comes and goes when heart beats...,I have been having panic attacks for a few mon...


In [5]:
test.head()

,dr_id,label,question_1,question_2
0,5,0,Had a cyst on my bottom. MRSA. Doc gave me dox...,Does being on Doxycycline for 10 days help cle...
1,2,0,Hello I have back spasms that flare up every n...,I have been having intense back spasms. Could ...
2,7,0,I having a flexible sigmoidoscopy and I have f...,I have a flexible sigmoidoscopy scheduled this...
3,8,1,Is it ok to induce your period while on the su...,Is it normal to get withdrawal bleeding while ...
4,3,0,Please recommend the best beer to prevent hang...,How to deal with hangover due to beer?


In [6]:
train.shape

(2438, 4)

In [7]:
test.shape

(610, 4)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [9]:
train["both_q"] = train["question_1"] +" "+ train["question_2"]

In [10]:
train.head()

,dr_id,label,question_1,question_2,both_q
0,3,1,Is it contraindicated to insert a urinary cath...,I am looking for some information- Is it safe ...,Is it contraindicated to insert a urinary cath...
1,8,1,How do I tell the difference between an allerg...,What are the symptoms that differentiate betwe...,How do I tell the difference between an allerg...
2,1,1,Can coarctation of the aorta cause poor growth...,"I am 4' 8"". My mom said that I have a birth de...",Can coarctation of the aorta cause poor growth...
3,9,0,I had a sinus infection 3 weeks ago and i was ...,My sinuses have been blocked due to allergies ...,I had a sinus infection 3 weeks ago and i was ...
4,9,0,Clicking sound comes and goes when heart beats...,I have been having panic attacks for a few mon...,Clicking sound comes and goes when heart beats...


In [11]:
vectorizer = TfidfVectorizer()
vectorizer.fit(train["both_q"])

train["q1_tfidf"] = vectorizer.transform(train["question_1"]).todense().tolist()
train["q2_tfidf"] = vectorizer.transform(train["question_2"]).todense().tolist()

train['q1_tfidf'] = train['q1_tfidf'].apply(lambda x: np.array(x))
train['q2_tfidf'] = train['q2_tfidf'].apply(lambda x: np.array(x))

In [12]:
train['cosineSim'] = train.apply(lambda x: cosine_similarity(x["q1_tfidf"].reshape(1, -1), x["q2_tfidf"].reshape(1, -1))[0][0], axis=1)

In [13]:
train.head()

,dr_id,label,question_1,question_2,both_q,q1_tfidf,q2_tfidf,cosineSim
0,3,1,Is it contraindicated to insert a urinary cath...,I am looking for some information- Is it safe ...,Is it contraindicated to insert a urinary cath...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.611276
1,8,1,How do I tell the difference between an allerg...,What are the symptoms that differentiate betwe...,How do I tell the difference between an allerg...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.607764
2,1,1,Can coarctation of the aorta cause poor growth...,"I am 4' 8"". My mom said that I have a birth de...",Can coarctation of the aorta cause poor growth...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.425709
3,9,0,I had a sinus infection 3 weeks ago and i was ...,My sinuses have been blocked due to allergies ...,I had a sinus infection 3 weeks ago and i was ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.025657
4,9,0,Clicking sound comes and goes when heart beats...,I have been having panic attacks for a few mon...,Clicking sound comes and goes when heart beats...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.115926


In [14]:
train["pred"] = (train["cosineSim"] >= 0.5).astype("int")

In [15]:
train.head()

,dr_id,label,question_1,question_2,both_q,q1_tfidf,q2_tfidf,cosineSim,pred
0,3,1,Is it contraindicated to insert a urinary cath...,I am looking for some information- Is it safe ...,Is it contraindicated to insert a urinary cath...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.611276,1
1,8,1,How do I tell the difference between an allerg...,What are the symptoms that differentiate betwe...,How do I tell the difference between an allerg...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.607764,1
2,1,1,Can coarctation of the aorta cause poor growth...,"I am 4' 8"". My mom said that I have a birth de...",Can coarctation of the aorta cause poor growth...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.425709,0
3,9,0,I had a sinus infection 3 weeks ago and i was ...,My sinuses have been blocked due to allergies ...,I had a sinus infection 3 weeks ago and i was ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.025657,0
4,9,0,Clicking sound comes and goes when heart beats...,I have been having panic attacks for a few mon...,Clicking sound comes and goes when heart beats...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.115926,0


In [18]:
def predict(df):
    df["q1_tfidf"] = vectorizer.transform(df["question_1"]).todense().tolist()
    df["q2_tfidf"] = vectorizer.transform(df["question_2"]).todense().tolist()

    df['q1_tfidf'] = df['q1_tfidf'].apply(lambda x: np.array(x))
    df['q2_tfidf'] = df['q2_tfidf'].apply(lambda x: np.array(x))
    df['cosineSim'] = df.apply(lambda x: cosine_similarity(x["q1_tfidf"].reshape(1, -1), x["q2_tfidf"].reshape(1, -1))[0][0], axis=1)
    df["pred"] = (df["cosineSim"] >= 0.5).astype("int")

In [19]:
predict(test)

In [25]:
from sklearn.metrics import classification_report
print(classification_report(test["label"], test["pred"]))

              precision    recall  f1-score   support

           0       0.63      0.88      0.73       312
           1       0.78      0.45      0.57       298

    accuracy                           0.67       610
   macro avg       0.70      0.66      0.65       610
weighted avg       0.70      0.67      0.65       610



In [53]:
train["combined_vecs"] = train.apply(lambda x: np.concatenate((x["q1_tfidf"], x["q2_tfidf"])), axis=1)
test["combined_vecs"] = train.apply(lambda x: np.concatenate((x["q1_tfidf"], x["q2_tfidf"])), axis=1)

In [59]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(np.stack(train["combined_vecs"].values), train["label"])
preds = clf.predict(np.stack(test["combined_vecs"].values))

In [62]:
print(classification_report(test["label"], preds))

              precision    recall  f1-score   support

           0       0.53      0.50      0.51       312
           1       0.51      0.54      0.52       298

    accuracy                           0.52       610
   macro avg       0.52      0.52      0.52       610
weighted avg       0.52      0.52      0.52       610



In [63]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
clf.fit(np.stack(train["combined_vecs"].values), train["label"])
preds = clf.predict(np.stack(test["combined_vecs"].values))

In [64]:
print(classification_report(test["label"], preds))

              precision    recall  f1-score   support

           0       0.52      0.47      0.49       312
           1       0.49      0.53      0.51       298

    accuracy                           0.50       610
   macro avg       0.50      0.50      0.50       610
weighted avg       0.50      0.50      0.50       610

